Experiment: POS baseline
------
uses a pos-baseline no attention scores necessaru

In [1]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

In [2]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)

06/04/2019 10:49:51 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
500it [00:00, 6849.54it/s]


In [3]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label'])

In [4]:
path_label_data = params.intermediary_task['task_specific_params']['target_data']
sentence_toks = prepare_model.get_sample_toks(path_label_data)

500it [00:00, 133194.79it/s]


In [5]:
indices = dataset.get_val('index')
intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = [label.flatten().tolist().index(1) for label in intermediary_labels]

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /sailhome/rdm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
pos_matrix = sentence_to_POS_matrix(sentence_toks, bias_indices, indices)

In [8]:
dataset.add_data(pos_matrix, "pos")

Run the cell below to reset params 
-----

In [25]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [26]:
print(params.final_task)

{'model': 'log_reg', 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


In [27]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='pos', label_key='bias_label', threshold=0.42)

In [29]:
statistics

{'auc': [(0.80824826748216, 0.9235432127396412), 0.86617697180165],
 'accuracy': [(0.7316964285714285, 0.8571428571428571), 0.7985714285714287]}

New run 

In [30]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [31]:
print(params.final_task)

{'model': 'log_reg', 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='pos', label_key='bias_label', threshold=0.42)

In [35]:
statistics

{'auc': [(0.8179396218542163, 0.9196035409902598), 0.8739939767877462],
 'accuracy': [(0.7363839285714285, 0.8482142857142858), 0.802142857142857]}

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [13]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
dataset.shuffle_data()
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [14]:
train_accs, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='pos', label_key='bias_label', threshold=0.42)

In [15]:
avg_train_acc = [average_data(accuracies) for accuracies in train_accs]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [18]:
min_train_acc, max_train_acc, avg_train_acc = get_statistics(avg_train_acc, "accuracy")

In [20]:
print(max_train_acc)

0.8390804597701149


In [22]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [23]:
print(max_auc)

0.8977896320471509


In [24]:
print(min_auc)

0.8517523777392199
